# Reboot of LCATS Story Analysis

## Imports

In [ ]:
from dataclasses import dataclass
from datetime import date
from typing import List, Dict, Callable, Optional

import json
import os
import re
import sys

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Third-party modules

In [ ]:
import dotenv
from openai import OpenAI
import tiktoken

Add imports from within the project

In [ ]:
# Add the parent directory to the path so we can import modules from the parent directory.
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from lcats import chunking
from lcats import extraction
from lcats import stories
from lcats import utils
from lcats.datasets import torchdata
from lcats.gatherers import extractors


## Project Setup

### Path Setup

In [ ]:
# If the following code is run from lcats/notebooks in VSCode and the data is in lcats/data ...
CURRENT_PATH = os.path.abspath(os.curdir)  # This is where the notebook is executing.
PROJECT_ROOT = os.path.dirname(CURRENT_PATH)   # This should be the root of the project.
DEV_CORPUS = os.path.abspath(os.path.join(PROJECT_ROOT, 'data'))  # Local copy of the data.
DEV_OUTPUT = os.path.abspath(os.path.join(PROJECT_ROOT, 'output'))  # Local copy of the data.
GIT_CORPUS = os.path.abspath(os.path.join(PROJECT_ROOT, '../corpora'))  # Data in the git repo.
OPENIA_API_KEYS_ENV = os.path.abspath(os.path.join(PROJECT_ROOT, '../.secrets/openai_api_keys.env'))  # Local OpenAI API key.

def check_path(path, description):
    if os.path.exists(path):
        print(f"Found {description} at: {path}")
    else:
        print(f"Missing {description} from: {path}")

check_path(DEV_CORPUS, "DEV_CORPUS")
check_path(DEV_OUTPUT, "DEV_OUTPUT")
check_path(GIT_CORPUS, "GIT_CORPUS")
check_path(OPENIA_API_KEYS_ENV, "OPENIA_API_KEYS_ENV")

## OpenAI Client

Get the OpenAI API key.

In [ ]:
dotenv.load_dotenv(OPENIA_API_KEYS_ENV)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
print(OPENAI_API_KEY)

Verify that we can get a client.

In [ ]:
client = OpenAI()
print(f"Loaded OpenAI client: {client} with version: {client._version}")

Verify the API is working. This week. And that you have credits.

In [ ]:
response = client.responses.create(
    model="gpt-4o",
    input="Write a one-sentence bedtime story about a starship captain visiting a planet."
)

print(f"Story generated on: {date.today()}:")
utils.pprint(response.output_text)

## Story Corpora

In [ ]:
# Reload the modules to ensure we have the latest code, if doing active development.
if False: 
    from importlib import reload
    reload(stories)
    reload(utils)

In [ ]:
# If run from within a notebook, the corpora root is two paths up from the notebook's location.
CORPORA_ROOT = GIT_CORPUS  # Checked-in corpora
# CORPORA_ROOT = DEV_CORPUS  # Command line working corpora

# Now load the corpora
corpora = stories.Corpora(CORPORA_ROOT)

print("Loaded corpora:")
print(f" - root: {corpora.corpora_root}")
print(f" - corpora: {len(corpora.corpora)}")
print(f" - stories: {len(corpora.stories)}")
print()
print(f"Example story: corpora.stories[0]:")
print(corpora.stories[0])

In [ ]:
example_story = corpora.stories[0]
print(f"Story type: {type(example_story)} with a body of {len(example_story.body)} characters.")

## Scene and Sequel Extraction

First-pass Prompts suggested by ChatGPT

In [ ]:
SCENE_SEQUEL_SYSTEM_PROMPT = """
You are a helpful assistant that breaks down stories into structured events.
Each event is labeled as "scene", "sequel", or "none" (if it doesn't fit exactly).
Follow these definitions:

- scene: a segment where a character with a goal attempts to achieve it, leading to success or disaster.
- sequel: a segment after a disaster or success, where a character reacts, processes emotions, considers options, and forms a new goal.

Your output MUST be valid JSON and only the JSON without any other text or comments.
"""

SCENE_SEQUEL_USER_PROMPT_TEMPLATE = """
I will give you a story in plain text.
1. Read the story carefully.
2. Identify major events or paragraphs that qualify as scenes or sequels (or 'none' if it doesn't clearly fit).
3. For each event, provide:
   - event_text: the text snippet or summary
   - event_type: 'scene' or 'sequel' or 'none'
   - reason: a short explanation of why you classified it that way
4. Return a JSON dictionary with one key named "events" - the output must be valid JSON and only the JSON.
Your output MUST be valid JSON and only the JSON without any other text or comments.

STORY:
\"\"\"{story_text}\"\"\"
"""

SCENE_SEQUEL_EXTRACTOR = extraction.ExtractionTemplate(
    name="scene_sequel_extractor",
    system_template=SCENE_SEQUEL_SYSTEM_PROMPT,
    user_template=SCENE_SEQUEL_USER_PROMPT_TEMPLATE)


In [ ]:
example_prompt = SCENE_SEQUEL_EXTRACTOR.build_prompt(example_story.body)
example_prompt

In [ ]:
def extract_scenes_and_sequels(
        story_text: str,        
        model_name: str = "gpt-4o",
        temperature: float = 0.2,
        template: Callable = SCENE_SEQUEL_EXTRACTOR,
    ) -> extraction.ExtractionResult:
    """
    Extract scenes and sequels from a story using the specified model.
    
    Args:
        story_text (str): The text of the story to analyze.
        model_name (str): The name of the OpenAI model to use for extraction.
    
    Returns:
        extraction.ExtractionResult: The result of the extraction, including parsed output and any errors.
    """
    return extraction.extract_from_story(
        story_text,
        template=template,
        client=client,
        model_name=model_name,
        temperature=temperature
    )


In [ ]:
result_gpt_35_turbo = extract_scenes_and_sequels(example_story.body, model_name="gpt-3.5-turbo")

In [ ]:
len(result_gpt_35_turbo.extracted_output), result_gpt_35_turbo.extracted_output[:5]

Note: this cell takes 30-90 seconds and is an expensive GPT 4.0 call.

In [ ]:
result_gpt_4o = extract_scenes_and_sequels(example_story.body, model_name="gpt-4o")

In [ ]:
len(result_gpt_4o.extracted_output), result_gpt_4o.extracted_output[:5]

### Save Extractions to Disk

In [ ]:
def make_serializable(result, nonserializable_key="response"):
    """
    Remove a non-serializable key from the result dictionary.

    Args:
        result (dict): The dictionary to clean.
        nonserializable_key (str): The key to remove if present.

    Returns:
        dict: A shallow copy of the dictionary with the specified key removed.
    """
    result = dict(result)  # shallow copy to avoid mutating original
    result.pop(nonserializable_key, None)
    return result

def extract_all_and_write(corpora, extractor, model_name, output_dir, file_namer, serializer):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Create a subdirectory for the model
    model_dir = os.path.join(output_dir, "scene_extraction", model_name)
    os.makedirs(model_dir, exist_ok=True)

    for story in corpora.stories:
        filename = file_namer(story.name) + "-scenes.json"
        filepath = os.path.join(model_dir, filename)

        # Skip already-processed files
        if os.path.exists(filepath):
            print(f"Skipping already processed story: {story.name}")
            continue

        try:
            print(f"Processing story: {story.name}")
            result = extractor(story.body, model_name=model_name)
            serialized_result = serializer(result)
            with open(filepath, "w") as f:
                json.dump(serialized_result, f, indent=2)
        except Exception as e:
            print(f"Error processing {story.name}: {e}")

    print("Scene extraction complete.")


### Extract Scenes for Two Popular Models

In [ ]:
extract_all_and_write(
    corpora,
    extract_scenes_and_sequels,
    model_name="gpt-4o",
    output_dir=DEV_OUTPUT,
    file_namer=extractors.title_to_filename,
    serializer=make_serializable)

In [ ]:
extract_all_and_write(
    corpora,
    extract_scenes_and_sequels,
    model_name="gpt-3.5-turbo",
    output_dir=DEV_OUTPUT,
    file_namer=extractors.title_to_filename,
    serializer=make_serializable)

### Fetch Extraction, Computing if Necessary

In [ ]:
def fetch_story_extraction(
    story,
    model_name,
    output_dir,
    file_namer
    ):
    # Determine file path
    model_dir = os.path.join(output_dir, "scene_extraction", model_name)
    os.makedirs(model_dir, exist_ok=True)
    filename = file_namer(story.name) + "-scenes.json"
    filepath = os.path.join(model_dir, filename)

    # If the file exists and we're not forcing recomputation, load and return
    if os.path.exists(filepath):
        with open(filepath, "r") as f:
            return json.load(f)
    return None

def fetch_or_compute_story_extraction(
    story,
    extractor,
    model_name,
    output_dir,
    file_namer,
    serializer,
    force=False
):
    """
    Fetch the scene/sequel extraction for a story, loading from disk if available,
    or computing and saving it otherwise.

    Parameters:
        story: A Story object with a .name and .body attribute.
        extractor: Function that extracts structured data from story.body.
        model_name: Name of the model used for the extraction (e.g., 'gpt-4o').
        output_dir: Root directory where extractions are stored.
        file_namer: Function to turn a story name into a safe filename.
        serializer: Function that removes or transforms non-serializable objects in the result.
        force: If True, reprocess the story even if a saved result exists.

    Returns:
        The structured extraction result (as loaded from JSON).
    """

    # Determine file path
    model_dir = os.path.join(output_dir, "scene_extraction", model_name)
    os.makedirs(model_dir, exist_ok=True)
    filename = file_namer(story.name) + "-scenes.json"
    filepath = os.path.join(model_dir, filename)

    # If the file exists and we're not forcing recomputation, load and return
    if not force and os.path.exists(filepath):
        with open(filepath, "r") as f:
            return json.load(f)

    # Otherwise compute, serialize, save, and return
    result = extractor(story.body, model_name=model_name)
    serializable_result = serializer(result)
    with open(filepath, "w") as f:
        json.dump(serializable_result, f, indent=2)
    return serializable_result


In [ ]:
example_story = corpora.stories[0]
example_extraction = fetch_story_extraction(
    example_story,
    model_name="gpt-4o",
    output_dir=DEV_OUTPUT,
    file_namer=extractors.title_to_filename,
)
len(example_extraction['parsed_output']['events'])

## Analyzing the First Pass Extraction

In [ ]:
def count_tokens(text: str, model: str = "gpt-3.5-turbo") -> int:
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

def collate_story_extractions(corpora, model_name, output_dir):
    """
    Collate the story extractions into a DataFrame.
    """
    extractions = {}
    statistics = []
    for story in corpora.stories:
        # Get metadata about the story
        story_text = story.body
        characters = len(story_text)
        tokens = count_tokens(story_text, model_name)

        # Get the extraction for the story
        extraction = fetch_story_extraction(
            story,
            model_name=model_name,
            output_dir=output_dir,
            file_namer=extractors.title_to_filename,
        )
        extractions[story.name] = extraction
        if extraction is None:
            # print(f"Missing extraction for story: {story.name}")
            parsed = False
            events = []
            scenes = []
            sequels = []
            nones = []
        else:
            # print(f"Processing extraction for story: {story.name}")
            parsed = True
            events = extraction['parsed_output']['events']
            scenes = [e for e in events if e['event_type'] == 'scene']
            sequels = [e for e in events if e['event_type'] == 'sequel']
            nones = [e for e in events if e['event_type'] == 'none']

        # Append the statistics for this story
        statistics.append({
            "story_name": story.name,
            "characters": characters,
            "tokens": tokens,
            "parsed": int(parsed),
            "events": len(events),
            "scenes": len(scenes),
            "sequels": len(sequels),
            "nones": len(nones)
        })
    return extractions, pd.DataFrame(statistics)



In [ ]:
gpt_4o_data, gpt_4o_df = collate_story_extractions(
    corpora,
    model_name="gpt-4o",
    output_dir=DEV_OUTPUT,
)
print(gpt_4o_df.describe())
gpt_4o_df.sum()

In [ ]:
gpt_35_data, gpt_35_df = collate_story_extractions(
    corpora,
    model_name="gpt-3.5-turbo",
    output_dir=DEV_OUTPUT,
)
print(gpt_35_df.describe())
gpt_35_df.sum()

In [ ]:
gpt_4o_df[gpt_4o_df["story_name"] == "Sherlock Holmes - The Adventure of the Engineer's Thumb"]["tokens"].values[0]

In [ ]:
comparisons = []
for story in corpora.stories:
    gpt_4o_extraction = gpt_4o_data[story.name]
    gpt_35_extraction = gpt_35_data[story.name]
    if gpt_4o_extraction is None or gpt_35_extraction is None:
        continue
    gpt_4o_tokens = gpt_4o_df[gpt_4o_df["story_name"] == story.name]['tokens'].values[0]
    gpt_35_tokens = gpt_35_df[gpt_35_df["story_name"] == story.name]['tokens'].values[0]
    gpt_4o_events = gpt_4o_extraction['parsed_output']['events']
    gpt_35_events = gpt_35_extraction['parsed_output']['events']
    gpt_4o_scenes = [e for e in gpt_4o_events if e['event_type'] == 'scene']
    gpt_35_scenes = [e for e in gpt_35_events if e['event_type'] == 'scene']
    gpt_4o_sequels = [e for e in gpt_4o_events if e['event_type'] == 'sequel']
    gpt_35_sequels = [e for e in gpt_35_events if e['event_type'] == 'sequel']
    gpt_4o_nones = [e for e in gpt_4o_events if e['event_type'] == 'none']
    gpt_35_nones = [e for e in gpt_35_events if e['event_type'] == 'none']
    print(f"Story: {story.name}")
    print(f" - characters: {len(story.body)}")
    print(f" - tokens: {gpt_4o_tokens} (gpt-4o) vs {gpt_35_tokens} (gpt-3.5-turbo)")
    print(f" - events: {len(gpt_4o_events)} (gpt-4o) vs {len(gpt_35_events)} (gpt-3.5-turbo)")
    print(f" - scenes: {len(gpt_4o_scenes)} (gpt-4o) vs {len(gpt_35_scenes)} (gpt-3.5-turbo)")
    print(f" - sequels: {len(gpt_4o_sequels)} (gpt-4o) vs {len(gpt_35_sequels)} (gpt-3.5-turbo)")
    print(f" - nones: {len(gpt_4o_nones)} (gpt-4o) vs {len(gpt_35_nones)} (gpt-3.5-turbo)")
    comparisons.append({
        "story_name": story.name,
        "model": "gpt-4o",
        "tokens": gpt_4o_tokens,
        "events": len(gpt_4o_events),
        "scenes": len(gpt_4o_scenes),
        "sequels": len(gpt_4o_sequels),
        "nones": len(gpt_4o_nones)
    })
    comparisons.append({
        "story_name": story.name,
        "model": "gpt-3.5-turbo",
        "tokens": gpt_35_tokens,
        "events": len(gpt_35_events),
        "scenes": len(gpt_35_scenes),
        "sequels": len(gpt_35_sequels),
        "nones": len(gpt_35_nones)
    })

comparison_df = pd.DataFrame(comparisons)

In [ ]:
print(comparison_df.columns)
print(comparison_df.describe())
comparison_df.head()

In [ ]:
common_4o = gpt_4o_df[gpt_35_df["parsed"] == 1]
common_35 = gpt_35_df[gpt_35_df["parsed"] == 1]
print("Common stories with valid extractions:")
print(f" - gpt-4o: {len(common_4o)}")
print(common_4o.describe())
print(f" - gpt-3.5-turbo: {len(common_35)}")
print(common_35.describe())


In [ ]:
gpt_4o_data[example_story.name]['parsed_output']['events'][:5]

In [ ]:
gpt_35_data[example_story.name]['parsed_output']['events'][:5]

In [ ]:
comparison_df.describe()

In [ ]:
def plot_columns_vs(df, x_col, y_col, hue_col=None):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df, x=x_col, y=y_col, hue=hue_col)
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    plt.title(f"{y_col} vs {x_col}")
    plt.legend(title=hue_col)
    plt.grid(True)
    plt.show()

plot_columns_vs(comparison_df, 'tokens', 'events', hue_col='model')


In [ ]:
plot_columns_vs(comparison_df, 'tokens', 'scenes', hue_col='model')


In [ ]:
plot_columns_vs(comparison_df, 'tokens', 'nones', hue_col='model')


## Pure Narrative Scene Extraction
Let's try just identifying the narrative scenes proper, regardless of whether they are dramatic scenes or sequels.

In [135]:
NARRATIVE_SCENE_SYSTEM_PROMPT = """You are a helpful assistant trained in analyzing narrative texts. Your task is to identify narrative scenes — contiguous segments of story that form coherent narrative units.

A **narrative scene** is a portion of text that is unified by time, space, characters, and action. Scenes usually involve the same location and characters, and follow continuous or causally related events, often sharing thematic or stylistic unity (dialogue, tone, etc.).

Scenes are separated by **narrative discontinuities**, such as shifts in time, space, character group, point of view, or event structure.

Use the following rules to help identify scenes:
- A scene generally takes place in one continuous time and place.
- A scene features a coherent group of characters engaged in related actions.
- A shift in time, location, or character group likely signals the end of a scene.
- Brief non-narrative passages (e.g. exposition or reflection) may be labeled as 'none' unless tightly integrated.

Your output must be valid JSON with one top-level key: `"events"`, containing a list of scenes or other segments.
"""

NARRATIVE_SCENE_USER_PROMPT = """
I will give you a story in plain text.

1. Read the story carefully.
2. Divide it into **narrative scenes**, using the definition and criteria provided.
3. For each segment, provide:
   - type: 'scene' if it is a narrative scene, 'none' if it is non-narrative material
   - text: the full passage that constitutes the segment
   - reason: a brief explanation of why it was classified this way
4. Return a JSON object with one key named "events", formatted as follows:

Example:
{{
  "events": [
    {{
      "type": "scene",
      "text": "John entered the room and picked up the phone...",
      "reason": "Continuous action in same room, same character, no time break."
    }},
    {{
      "type": "none",
      "text": "It had been three years since the war...",
      "reason": "Expository flashback without active characters or events."
    }}
  ]
}}

Return only the JSON.

STORY:
\"\"\"{story_text}\"\"\"
"""
NARRATIVE_SCENE_EXTRACTOR = extraction.ExtractionTemplate(
    name="narrative_scene_extractor",
    system_template=NARRATIVE_SCENE_SYSTEM_PROMPT,
    user_template=NARRATIVE_SCENE_USER_PROMPT,
)

def extract_narrative_scenes(
        story_text: str,        
        model_name: str = "gpt-4o",
        temperature: float = 0.2,
        template: Callable = NARRATIVE_SCENE_EXTRACTOR,
    ) -> extraction.ExtractionResult:
    """
    Extract narrative scenes from a story using the specified model.
    
    Args:
        story_text (str): The text of the story to analyze.
        model_name (str): The name of the OpenAI model to use for extraction.
    
    Returns:
        extraction.ExtractionResult: The result of the extraction, including parsed output and any errors.
    """
    return extraction.extract_from_story(
        story_text,
        template=template,
        client=client,
        model_name=model_name,
        temperature=temperature
    )



Verify the new version is working.

In [136]:
narrative_scene_gpt_35_turbo = extract_narrative_scenes(example_story.body, model_name="gpt-3.5-turbo")
len(narrative_scene_gpt_35_turbo.extracted_output), narrative_scene_gpt_35_turbo.extracted_output[:5]


(8,
 [{'type': 'none',
   'text': 'Of all the problems which have been submitted to my friend, Mr. Sherlock Holmes, for solution during the years of our intimacy, there were only two which I was the means of introducing to his notice—that of Mr. Hatherley’s thumb, and that of Colonel Warburton’s madness.',
   'reason': 'Introduction and exposition, no active characters or events.'},
  {'type': 'scene',
   'text': 'It was in the summer of ’89, not long after my marriage, that the events occurred which I am now about to summarise.',
   'reason': 'Introduction of time setting and events about to be summarized.'},
  {'type': 'scene',
   'text': 'One morning, at a little before seven o’clock, I was awakened by the maid tapping at the door to announce that two men had come from Paddington and were waiting in the consulting-room.',
   'reason': 'Introduction of a new event with characters in a specific location.'},
  {'type': 'scene',
   'text': 'I entered my consulting-room and found a gentl

Note: this cell takes roughly 2-3 minutes and is an expensive GPT 4.0 call.

In [137]:
narrative_scene_gpt_4o = extract_narrative_scenes(example_story.body, model_name="gpt-4o")
len(narrative_scene_gpt_4o.extracted_output), narrative_scene_gpt_4o.extracted_output[:5]

(18,
 [{'type': 'none',
   'text': 'Of all the problems\nwhich have been submitted to my friend, Mr. Sherlock Holmes, for solution\nduring the years of our intimacy, there were only two which I was the means of\nintroducing to his notice—that of Mr. Hatherley’s thumb, and that\nof Colonel Warburton’s madness. Of these the latter may have afforded a\nfiner field for an acute and original observer, but the other was so strange in\nits inception and so dramatic in its details that it may be the more worthy of\nbeing placed upon record, even if it gave my friend fewer openings for those\ndeductive methods of reasoning by which he achieved such remarkable results.\nThe story has, I believe, been told more than once in the newspapers, but, like\nall such narratives, its effect is much less striking when set forth en\nbloc in a single half-column of print than when the facts slowly evolve\nbefore your own eyes, and the mystery clears gradually away as each new\ndiscovery furnishes a step whic

## Chunk-Based Extraction

In [ ]:
# Max tokens divided by min scenes gives us a rough estimate of how many tokens are needed per scene.
estimated_tokens_per_scene = float(comparison_df['tokens'].max() / comparison_df['scenes'].min())
estimated_tokens_per_scene


In [ ]:
example_story = corpora.stories[0]
story_text = example_story.body
chunks = chunking.chunk_story(story_text, max_tokens=6000, overlap_tokens=0, model_name="gpt-3.5-turbo")

print(f"Chunked story into {len(chunks)} parts.")
for index, chunk in enumerate(chunks):
    print(f"Chunk {index} ({len(chunk.text)} chars):")
    print(chunk.text[:100] + "...")
    print()

Chunking with overlap is likely smarter.

In [ ]:
example_story = corpora.stories[0]
example_text = example_story.body

example_chunks = chunking.chunk_story(example_text, max_tokens=6000, overlap_tokens=200, model_name="gpt-3.5-turbo")
chunking.display_chunks(example_chunks)

### Extracting Scenes - LASTREVIEW

In [ ]:
@dataclass
class SceneSpan:
    chunk_index: int
    relative_start: int
    relative_end: int
    scene_type: str
    text: str
    offset: int  # character offset in original story

    def absolute_start(self) -> int:
        return self.offset + self.relative_start

    def absolute_end(self) -> int:
        return self.offset + self.relative_end

SCENE_SEQUEL_SYSTEM_PROMPT = """
You are a helpful assistant that breaks down stories into structured narrative events.

Each event must be labeled as:
- "scene": a segment where a character pursues a goal, leading to success or failure.
- "sequel": a segment where a character reacts to the outcome, reflects, and formulates a new plan.
- "none": a segment that does not clearly fit into either category (e.g., exposition or transition).

You MUST output a list of events as a JSON dictionary with one key: "events".

Each event must include:
- "type": one of "scene", "sequel", or "none"
- "reason": a brief explanation of why you classified it this way
- "text": the exact event text, copied from the story
- "start_char": the starting character offset of the event in the original story
- "end_char": the ending character offset (exclusive)

Do NOT include any additional explanation or formatting.
Only return valid JSON.
"""

SCENE_SEQUEL_USER_PROMPT_TEMPLATE = """
I will give you a story in plain text.

Please identify contiguous narrative segments that represent:
- scenes (goal-driven attempts)
- sequels (reflections after outcomes)
- or neither ("none" for other material)

For each segment, return a dictionary with:
- "type": "scene", "sequel", or "none"
- "reason": a brief explanation of why you classified it this way
- "text": the exact span of text from the story
- "start_char": starting character index of this span in the story
- "end_char": ending character index (exclusive)

Return a single JSON object with a key named "events" and a list of these entries.

The output MUST be valid JSON and include ONLY the JSON, no explanation or comments.

STORY:
\"\"\"{story_text}\"\"\"
"""

def build_scene_sequel_prompt(story_text: str) -> list:
    return [
        {"role": "system", "content": SCENE_SEQUEL_SYSTEM_PROMPT},
        {"role": "user", "content": SCENE_SEQUEL_USER_PROMPT_TEMPLATE.format(story_text=story_text)}
    ]


def extract_scenes_and_sequels(story_text: str, model_name="gpt-3.5-turbo") -> Dict:
    messages = build_scene_sequel_prompt(story_text)

    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        temperature=0.2,
    )

    raw_output = response.choices[0].message.content

    try:
        parsed_output = utils.extract_json(raw_output)
        parsing_error = None
    except json.JSONDecodeError as exc:
        parsed_output = None
        parsing_error = str(exc)

    scene_spans: List[SceneSpan] = []
    extraction_error = None

    if isinstance(parsed_output, dict) and "events" in parsed_output:
        for i, event in enumerate(parsed_output["events"]):
            scene_text = event.get("text", "").strip()
            scene_type = event.get("type", "scene")

            # Match scene_text in story_text to find offsets
            match = re.search(re.escape(scene_text), story_text)
            if match:
                start_char = match.start()
                end_char = match.end()
            else:
                start_char = None
                end_char = None

            scene_spans.append(SceneSpan(
                chunk_index=0,  # Actual index will be set later by tail extraction logic
                relative_start=0,  # Will be calculated by offset adjustment later
                relative_end=0,
                scene_type=scene_type,
                text=scene_text,
                offset=start_char if start_char is not None else 0  # fallback
            ))

            # Patch relative positions if match succeeded
            if match:
                scene_spans[-1].relative_start = 0  # initially 0, will be corrected in stitching
                scene_spans[-1].relative_end = end_char - start_char

    else:
        scene_spans = []
        extraction_error = "Expected 'events' key in JSON response."

    return {
        "story_text": story_text,
        "model_name": model_name,
        "messages": messages,
        "response": response,
        "raw_output": raw_output,
        "parsed_output": parsed_output,
        "extracted_output": scene_spans,
        "parsing_error": parsing_error,
        "extraction_error": extraction_error,
    }


def span_is_complete(span: Dict) -> bool:
    """
    Simple heuristic to determine if a scene span is complete.
    """
    text = span.get("text", "").strip()
    return text.endswith(".") and not text.endswith("...") and len(text) > 20

def get_span_from_raw(
    raw: Dict,
    chunk_index: int,
    chunk_input_text: str,
    carryover_len: int,
    chunk_offset: int
) -> Optional[SceneSpan]:
    """
    Convert a raw dict (LLM output) into a SceneSpan object.
    Uses fallback matching if start_char and end_char are missing.
    """
    scene_text = raw.get("text", "").strip()
    scene_type = raw.get("type", "scene")

    start_char = raw.get("start_char")
    end_char = raw.get("end_char")

    if start_char is None or end_char is None:
        match = re.search(re.escape(scene_text), chunk_input_text)
        if not match:
            return None
        start_char = match.start()
        end_char = match.end()

    adjusted_start = max(0, start_char - carryover_len)
    adjusted_end = max(0, end_char - carryover_len)
    print(f"Processing span: {scene_text} (start: {start_char}, end: {end_char}) "
          f"-> adjusted start: {adjusted_start}, adjusted end: {adjusted_end}")
    if adjusted_end <= 0:
        print(f"Skipping span entirely in carryover: {scene_text}")
        return None  # entirely in carryover, skip

    return SceneSpan(
        chunk_index=chunk_index,
        relative_start=adjusted_start,
        relative_end=adjusted_end,
        scene_type=scene_type,
        text=chunk_input_text[start_char:end_char],
        offset=chunk_offset
    )

def extract_carryover_text(
    raw_span: Dict,
    chunk_input_text: str
) -> str:
    """
    Extract tail of the final scene span for prepending to the next chunk.
    Falls back on re-matching if no indices are given.
    """
    last_text = raw_span.get("text", "")
    start = raw_span.get("start_char")
    end = raw_span.get("end_char")

    if start is not None and end is not None:
        return chunk_input_text[start:end]

    match = re.search(re.escape(last_text), chunk_input_text)
    return chunk_input_text[match.start():match.end()] if match else ""


def extract_scene_spans_with_tail(
    chunks: List[Chunk],
    model_name: str,
    extract_fn: Callable[[str, str], Dict]
) -> List[SceneSpan]:
    stitched_spans: List[SceneSpan] = []
    carryover_text = ""

    for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i + 1}/{len(chunks)}: {len(chunks[i].text)} chars")
        chunk_input_text = carryover_text + chunk.text
        carryover_len = len(carryover_text)

        # Call the model extractor
        result = extract_fn(chunk_input_text, model_name=model_name)
        raw_spans = result.get("extracted_output", [])
        print(f"Extracted {len(raw_spans)} spans from chunk {i + 1}")

        # Try to convert all spans to SceneSpan
        for raw in raw_spans:
            span = get_span_from_raw(
                raw,
                chunk_index=i,
                chunk_input_text=chunk_input_text,
                carryover_len=carryover_len,
                chunk_offset=chunk.start_char
            )
            if span:
                stitched_spans.append(span)

        # Setup carryover if final span is incomplete
        if raw_spans and not span_is_complete(raw_spans[-1]):
            carryover_text = extract_carryover_text(raw_spans[-1], chunk_input_text)
        else:
            carryover_text = ""

    return stitched_spans


In [ ]:
example_chunk = example_chunks[0]
example_text = example_chunk.text
example_prompt = build_scene_sequel_prompt(example_text)
example_result = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=example_prompt,
    temperature=0.2,
)
example_raw_output = example_result.choices[0].message.content
example_parsed_output = utils.extract_json(example_raw_output)
example_spans = example_parsed_output["events"]

In [ ]:
len(example_spans)

In [ ]:
example_spans[1]

In [ ]:
def summarize(text, length=72):
    """
    Summarize the text to a specified length.
    """
    if len(text) <= length:
        return text
    prefix = length // 2
    return text[:prefix] + "..." + text[-(length - prefix):]


for i, span in enumerate(example_spans):
    # print(span)
    print(f"Span {i} type: {span.get('type')}, text: {summarize(span.get('text', ''))}")
    start_char = span.get("start_char")
    end_char = span.get("end_char")
    if start_char is not None and end_char is not None:
        span_text = example_text[start_char:end_char]
        print(f" - start: {start_char}, end: {end_char}, text: {summarize(span_text)}...")


In [ ]:
example_chunk = example_chunks[0]
extraction = extract_scenes_and_sequels(
    example_chunk.text, model_name="gpt-3.5-turbo")
extraction['extracted_output'][:5]

In [ ]:
extract_scene_spans_with_tail(
    example_chunks,
    model_name="gpt-3.5-turbo",
    extract_fn=extract_scenes_and_sequels
)

In [ ]:
len(chunks), chunks[0][:100], chunks[-1][:100]

In [ ]:
for index, chunk in enumerate(chunks):
    print(f"Chunk {index} ({len(chunk)} chars):")
    print(chunk[:100] + "...")
    print()